<a href="https://colab.research.google.com/github/crossorbit/hgmldl/blob/main/part6_02_02_Improving_Perfomance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Augmentation

#### Albumentations
- https://github.com/albumentations-team/albumentations
- Albumentations: Fast and Flexible Image Augmentations 
- 빠르고 직관적이며 sequential하게 데이터 augmentation을 할 수 있도록 도와주는 라이브러리
- 기존 이미지 데이터에서 새로운 학습 데이터를 생성하기 위해 70가지 이상의 다양한 augmentation기능을 갖추고 있음
- 요즘 거의 다 사용함

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
sys.path.append("/content/drive/MyDrive/#fastcampus")

Mounted at /content/drive


In [ ]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/#fastcampus']

In [ ]:
#!pip install albumentations

In [ ]:
import os
import math
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

os.environ['CUDA_VISIBLE_DEVICES'] = '1' # 특정 GPU만 사용하도록 제한. "GPU 1번 만 사용하겠습니다"

In [ ]:
import albumentations as A
import cv2

# Augmentation 은 주로 트레이닝 단계에서 사용 함
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            # Declare an augmentation pipeline
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5), # 좌우반전. p는 해당 변환을 적용할 확률
                A.ShiftScaleRotate( #상화좌우 이동, 회전 등
                    p=0.5,
                    shift_limit=0.05, # 5% 넘지 않도록 제한
                    scale_limit=0.05,
                    rotate_limit=15, # 15도 회전
                ),
                # A.CoarseDropout( # 사각형 구멍 송송. 이미지 학습에 자주 사용
                #     p=0.5,
                #     max_holes=8,
                #     max_height=int(0.1 * size),
                #     max_width=int(0.1 * size)
                # ),
                A.RandomBrightnessContrast(p=0.2),
            ])
    def __call__(self, **kwargs):
        if self.transform:
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

In [ ]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, batch_size, csv_path, image_size, fold, mode='train', shuffle=True):
    self.fold = fold
    self.shuffle = shuffle
    self.mode = mode
    self.batch_size = batch_size
    self.image_size = image_size

    self.df = pd.read_csv(csv_path)
    if self.mode == 'train':
      self.df = self.df[self.df['fold'] != self.fold]
    elif self.mode == 'val':
      self.df = self.df[self.df['fold'] == self.fold]


    #### Remove invalid files
    #### https://github.com/tensorflow/models/issues/3134
    invalid_filenames = [
        'Egyptian_Mau_14',
        'Egyptian_Mau_139',
        'Egyptian_Mau_145',
        'Egyptian_Mau_156',
        'Egyptian_Mau_167',
        'Egyptian_Mau_177',
        'Egyptian_Mau_186',
        'Egyptian_Mau_191',
        'Abyssinian_5',
        'Abyssinian_34',
        'chihuahua_121',
        'beagle_116'
    ]
    self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]

    self.transform = Augmentation(image_size, mode)
    
    self.on_epoch_end()
          
  def __len__(self):
      return math.ceil(len(self.df) / self.batch_size)
  
  def __getitem__(self, idx):
      strt = idx * self.batch_size
      fin = (idx + 1) * self.batch_size
      data = self.df.iloc[strt:fin]
      
      batch_x, batch_y = self.get_data(data)

      return np.array(batch_x), np.array(batch_y)
      
  def get_data(self, data):
      batch_x = []
      batch_y = []
  
      for _, r in data.iterrows():
          file_name = r['file_name']

          image = cv2.imread(f'/content/drive/MyDrive/#fastcampus/images/{file_name}.jpg')
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image = cv2.resize(image, (self.image_size, self.image_size))
          
          #이미지 불러온 후 바로 transform 적용
          if self.mode == 'train':
              image = image.astype('uint8') #uint8 일 때만 적용되는 Augmentation이 있어서 변환
              image = self.transform(image=image)
              
          image = image.astype('float32')
          image = image / 255.

          label = int(r['species']) - 1

          batch_x.append(image)
          batch_y.append(label)
      
      return batch_x, batch_y
      
  def on_epoch_end(self):
      if self.shuffle:
          self.df = self.df.sample(frac=1).reset_index(drop=True)
          
csv_path = '/content/drive/MyDrive/#fastcampus/kfolds.csv'
train_generator = DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True)

valid_generator = DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True)

In [ ]:
class_name = ['Cat', 'Dog']

for batch in train_generator:
  X, y  = batch
  plt.figure(figsize=(15, 15))

  for i in range(9):
    ax = plt.subplot(3, 3, i+1) # 3*3 plot 으로 인덱스에 따라 결과 출력
    plt.imshow(X[i])
    plt.title(class_name[y[i]])
    plt.axis('off')

  break # 첫번쨰 배치 확인 후 종료

In [ ]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # Input
            layers.Input(input_shape),

            # 1st Conv block
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),

            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
        
            # Classfier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ]
    )

    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     7

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

## Transfer Learning
- https://github.com/keras-team/keras-applications
- 처음부터 시작하는 것 보다는 pre-trainig 모델을 활용하여 전이학습으로 시작하는 것을 추천
- accuracy 와 size 고려하여 선택

In [6]:
import os
import math
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [9]:
import albumentations as A
import cv2

# Augmentation 은 주로 트레이닝 단계에서 사용 함
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            # Declare an augmentation pipeline
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5), # 좌우반전. p는 해당 변환을 적용할 확률
                A.ShiftScaleRotate( #상화좌우 이동, 회전 등
                    p=0.5,
                    shift_limit=0.05, # 5% 넘지 않도록 제한
                    scale_limit=0.05,
                    rotate_limit=15, # 15도 회전
                ),
                #  A.CoarseDropout( # 사각형 구멍 송송. 이미지 학습에 자주 사용
                #      p=0.5,
                #      max_holes=8,
                #      max_height=int(0.1 * size),
                #      max_width=int(0.1 * size)
                # ),
                A.RandomBrightnessContrast(p=0.2),
            ])
    def __call__(self, **kwargs):
        if self.transform:
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

In [10]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, batch_size, csv_path, image_size, fold, mode='train', shuffle=True):
    self.fold = fold
    self.shuffle = shuffle
    self.mode = mode
    self.batch_size = batch_size
    self.image_size = image_size

    self.df = pd.read_csv(csv_path)
    if self.mode == 'train':
      self.df = self.df[self.df['fold'] != self.fold]
    elif self.mode == 'val':
      self.df = self.df[self.df['fold'] == self.fold]


    #### Remove invalid files
    #### https://github.com/tensorflow/models/issues/3134
    invalid_filenames = [
        'Egyptian_Mau_14',
        'Egyptian_Mau_139',
        'Egyptian_Mau_145',
        'Egyptian_Mau_156',
        'Egyptian_Mau_167',
        'Egyptian_Mau_177',
        'Egyptian_Mau_186',
        'Egyptian_Mau_191',
        'Abyssinian_5',
        'Abyssinian_34',
        'chihuahua_121',
        'beagle_116'
    ]
    self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]

    self.transform = Augmentation(image_size, mode)
    
    self.on_epoch_end()
          
  def __len__(self):
      return math.ceil(len(self.df) / self.batch_size)
  
  def __getitem__(self, idx):
      strt = idx * self.batch_size
      fin = (idx + 1) * self.batch_size
      data = self.df.iloc[strt:fin]
      
      batch_x, batch_y = self.get_data(data)

      return np.array(batch_x), np.array(batch_y)
      
  def get_data(self, data):
      batch_x = []
      batch_y = []
  
      for _, r in data.iterrows():
          file_name = r['file_name']

          image = cv2.imread(f'/content/drive/MyDrive/#fastcampus/images/{file_name}.jpg')
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image = cv2.resize(image, (self.image_size, self.image_size))
          
          #이미지 불러온 후 바로 transform 적용
          if self.mode == 'train':
              image = image.astype('uint8') #uint8 일 때만 적용되는 Augmentation이 있어서 변환
              image = self.transform(image=image)
              
          image = image.astype('float32')
          image = image / 255.

          label = int(r['species']) - 1

          batch_x.append(image)
          batch_y.append(label)
      
      return batch_x, batch_y
      
  def on_epoch_end(self):
      if self.shuffle:
          self.df = self.df.sample(frac=1).reset_index(drop=True)
          
csv_path = '/content/drive/MyDrive/#fastcampus/kfolds.csv'
train_generator = DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True)

valid_generator = DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True)

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0

def get_model(input_shape):
    inputs = keras.Input(input_shape)
    base_model = EfficientNetB0(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg'
    )
    
    x = base_model(inputs)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, output)
    
    return model

input_shape = (256, 256, 3)
model = get_model(input_shape)

# transfer Learning 시에는 Learning Rate 는 작게 적용
adam = keras.optimizers.Adam(lr=0.0001)

model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

16719872/16705208 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

Epoch 1/10


In [ ]:
  import matplotlib.pyplot as plt
history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()